In [18]:
#Import Packages

import pandas as pd
import seaborn as sns
%matplotlib inline
import nfl_data_py as nfl
import time
import cfbd
from cfbd.rest import ApiException
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [27]:
#Specifying Year Range
years = range(2014,2025)

In [49]:
#Pull in CSVs from Personal GitHubs
df_defensive = pd.read_csv("https://raw.githubusercontent.com/JoshKepler/Portfolio/main/NFL%20Project/Data/defensive_stats.csv")

power_5_team  = pd.read_csv("https://raw.githubusercontent.com/JoshKepler/Portfolio/main/NFL%20Project/Data/power_5_teams.csv")

In [50]:
#Create Defensive Career DF

#Fill NAs and combine relevant stats
df_defensive_career = df_defensive.fillna(0)
df_defensive_career = df_defensive_career.groupby(['player_id','player', 'position']).agg({'team_name':'last', 'Season':'last', 'player_game_count':'sum', 'snap_counts_defense':'sum', 'snap_counts_run_defense':'sum', 'snap_counts_coverage':'sum', 'snap_counts_slot':'sum', 'grades_defense': 'max', 'tackles':'sum', 'assists':'sum', 'missed_tackles': 'sum', 'stops':'sum', 'forced_fumbles':'sum', 'grades_tackle':'max', 'grades_run_defense': 'max', 'total_pressures': 'sum','hurries': 'sum', 'hits':'sum', 'sacks':'sum', 'grades_pass_rush_defense': 'max', 'targets':'sum', 'batted_passes':'sum', 'interceptions':'sum', 'receptions':'sum', 'yards':'sum', 'yards_after_catch':'sum', 'touchdowns':'sum', 'qb_rating_against': 'mean', 'grades_coverage_defense':'max'})
seasons_played = df_defensive.groupby(['player_id','player', 'position']).agg({'Season':'count'})

#DF data munging
df_defensive_career['seasons_played'] = seasons_played['Season']
df_defensive_career['draft_class'] = df_defensive_career['Season']+1
df_defensive_career = df_defensive_career.reset_index()
df_defensive_career['player'] = df_defensive_career['player'].str.replace(r'\.?\s*Jr\.?', '', regex=True)
df_defensive_career['player_key'] = df_defensive_career['player'] + '_' + df_defensive_career['team_name'].str.lower()
df_defensive_career = df_defensive_career.merge(power_5_team, how='left', on='team_name')

#DF result
df_defensive_career

,player_id,player,position,team_name,Season,player_game_count,snap_counts_defense,snap_counts_run_defense,snap_counts_coverage,snap_counts_slot,...,receptions,yards,yards_after_catch,touchdowns,qb_rating_against,grades_coverage_defense,seasons_played,draft_class,player_key,power_5_team?
0,9436,Dante Fowler,ED,FLORIDA,2016,12,665,281,59,21,...,2,2,3,0,70.1,71.6,1.0,2017,Dante Fowler_florida,1
1,9439,Leonard Williams,DI,USC,2016,13,857,330,0,0,...,0,0,0,0,0.0,68.8,1.0,2017,Leonard Williams_usc,1
2,9441,Vic Beasley,ED,CLEMSON,2016,13,561,238,8,0,...,1,-5,3,0,79.2,59.8,1.0,2017,Vic Beasley_clemson,1
3,9444,Trae Waynes,CB,MICH STATE,2016,13,734,266,453,22,...,30,446,171,1,60.4,79.4,1.0,2017,Trae Waynes_mich state,1
4,9445,Danny Shelton,DI,WASHINGTON,2016,14,903,363,3,0,...,0,0,0,0,0.0,78.9,1.0,2017,Danny Shelton_washington,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24160,182912,Chris Peal,CB,GEORGIA,2023,1,6,5,1,1,...,0,0,0,0,0.0,60.0,1.0,2024,Chris Peal_georgia,1
24161,182947,Unknown Temple D99,ED,TEMPLE,2023,1,6,3,0,0,...,0,0,0,0,0.0,0.0,1.0,2024,Unknown Temple D99_temple,0
24162,183045,Joshua Valdez-Alaniz,ED,MARSHALL,2023,1,7,0,0,0,...,0,0,0,0,0.0,0.0,1.0,2024,Joshua Valdez-Alaniz_marshall,0
24163,183052,Marlon McClendon,CB,GA STATE,2023,1,13,2,11,1,...,0,0,0,0,0.0,46.9,1.0,2024,Marlon McClendon_ga state,0


In [51]:
#Create final season DF

#Fill NAs and combine relevant stats
df_defensive_last_season = df_defensive.fillna(0)
df_defensive_last_season = df_defensive_career.groupby(['player_id','player']).last()
df_defensive_last_season = df_defensive_last_season.reset_index()

#DF Data mungering
df_defense_players = df_defensive_career.merge(df_defensive_last_season, on='player_id', suffixes=('','_last_season'))
df_defense_players = df_defense_players.drop(['seasons_played_last_season', 'draft_class_last_season', 'player_key_last_season'], axis=1)
df_defense_players = df_defense_players.groupby(['player_id']).last()
df_defense_players = df_defense_players.reset_index()

#DF result
df_defense_players

,player_id,player,position,team_name,Season,player_game_count,snap_counts_defense,snap_counts_run_defense,snap_counts_coverage,snap_counts_slot,...,targets_last_season,batted_passes_last_season,interceptions_last_season,receptions_last_season,yards_last_season,yards_after_catch_last_season,touchdowns_last_season,qb_rating_against_last_season,grades_coverage_defense_last_season,power_5_team?_last_season
0,9436,Dante Fowler,ED,FLORIDA,2016,12,665,281,59,21,...,3,2,0,2,2,3,0,70.1,71.6,1
1,9439,Leonard Williams,DI,USC,2016,13,857,330,0,0,...,0,3,1,0,0,0,0,0.0,68.8,1
2,9441,Vic Beasley,ED,CLEMSON,2016,13,561,238,8,0,...,1,2,0,1,-5,3,0,79.2,59.8,1
3,9444,Trae Waynes,CB,MICH STATE,2016,13,734,266,453,22,...,59,0,3,30,446,171,1,60.4,79.4,1
4,9445,Danny Shelton,DI,WASHINGTON,2016,14,903,363,3,0,...,0,0,0,0,0,0,0,0.0,78.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20518,182912,Chris Peal,CB,GEORGIA,2023,1,6,5,1,1,...,0,0,0,0,0,0,0,0.0,60.0,1
20519,182947,Unknown Temple D99,ED,TEMPLE,2023,1,6,3,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
20520,183045,Joshua Valdez-Alaniz,ED,MARSHALL,2023,1,7,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
20521,183052,Marlon McClendon,CB,GA STATE,2023,1,13,2,11,1,...,0,0,0,0,0,0,0,0.0,46.9,0


In [161]:
#importing combine & draft dfs from API
combine_df  = nfl.import_combine_data(years)
draft_df = nfl.import_draft_picks(years)

In [162]:
#Data Mungering
combine_df['player_name'] = combine_df['player_name'].str.replace(r'\.?\s*Jr\.?', '', regex=True) #Removing the jr. prefix
combine_df['player_key'] = combine_df['player_name'] + '_' + combine_df['school'].str.lower() #Creating player key
combine_df = combine_df.sort_values(by='ht')
combine_df = combine_df[:-29]

def extract_height(height_str):
    # Extract feet and inches from the string
    feet, inches = height_str.split('-')
    return int(feet) * 12 + int(inches)

# Apply the function to create a new 'ht_inches' column
combine_df['ht'] = combine_df['ht'].apply(extract_height)

#Combine DF result
combine_df.sort_values(by='ht')#.tail(29)

,season,draft_year,draft_team,draft_round,draft_ovr,pfr_id,cfb_id,player_name,pos,school,ht,wt,forty,bench,vertical,broad_jump,cone,shuttle,player_key
7138,2021,NaN,None,NaN,NaN,None,blake-lynch-2,Blake Lynch,K,Kansas St.,64,144.0,NaN,NaN,NaN,NaN,NaN,NaN,Blake Lynch_kansas st.
6849,2020,NaN,None,NaN,NaN,TaylJJ01,jj-taylor-1,J.J. Taylor,RB,Arizona,65,185.0,4.61,19.0,34.5,118.0,7.00,4.15,J.J. Taylor_arizona
7969,2023,2023.0,Dallas Cowboys,6.0,212.0,VaugDe00,deuce-vaughn-1,Deuce Vaughn,RB,Kansas St.,65,179.0,NaN,NaN,35.5,116.0,NaN,NaN,Deuce Vaughn_kansas st.
7205,2021,NaN,None,NaN,NaN,None,jarrett-patterson-1,Jarrett Patterson,OL,Notre Dame,66,195.0,4.52,NaN,NaN,NaN,NaN,NaN,Jarrett Patterson_notre dame
7102,2021,NaN,None,NaN,NaN,None,jamal-jones-4,Jamal Jones,RB,Arkansas St.,66,198.0,4.67,23.0,28.0,112.0,7.39,4.65,Jamal Jones_arkansas st.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8196,2024,NaN,None,NaN,NaN,None,amarius-mims-1,Amarius Mims,OT,Georgia,80,340.0,5.07,NaN,25.5,111.0,NaN,NaN,Amarius Mims_georgia
8218,2024,NaN,None,NaN,NaN,None,patrick-paul-1,Patrick Paul,OT,Houston,80,331.0,5.13,NaN,29.0,NaN,7.65,NaN,Patrick Paul_houston
6086,2018,2018.0,Oakland Raiders,1.0,15.0,MillKo00,kolton-miller-1,Kolton Miller,OT,UCLA,81,309.0,4.95,24.0,31.5,121.0,7.34,4.49,Kolton Miller_ucla
8004,2024,NaN,None,NaN,NaN,None,joe-alt-1,Joe Alt,OT,Notre Dame,81,321.0,5.05,NaN,28.0,112.0,7.31,4.51,Joe Alt_notre dame


In [163]:
#Merging College Stats with Combine Measurements
prospect_df = combine_df.merge(df_defense_players, on='player_key')

#Filtering by Defensive Postions
defensive_pos = ['CB', 'ED', 'LB', 'DI', 'S']
prospect_df = prospect_df[prospect_df['position'].isin(defensive_pos)]
prospect_df['pos'] = prospect_df['position']

In [164]:
#Data Mungering
prospect_df = prospect_df[prospect_df['season'] != 2024] #as of 5/6/24 2024 data was blank. Will add later to make perdictions
prospect_df['draft_ovr'] = prospect_df['draft_ovr'].fillna(256) #pick 256 will represent undrafted
prospect_df['draft_round'] = prospect_df['draft_round'].fillna(8) #round 8 will represent undrafted
prospect_df['draft_team'] = prospect_df['draft_team'].fillna('Undrafted')
prospect_df['draft_year'] = prospect_df['season']

In [165]:
#Data into positional buckets
prospect_cb_df = prospect_df[prospect_df['pos'] == 'CB']
prospect_ed_df = prospect_df[prospect_df['pos'] == 'ED']
prospect_lb_df = prospect_df[prospect_df['pos'] == 'LB']
prospect_di_df = prospect_df[prospect_df['pos'] == 'DI']
prospect_s_df = prospect_df[prospect_df['pos'] == 'S']

pos_df_dict = {'prospect_cb_df':prospect_cb_df , 'prospect_ed_df':prospect_ed_df, 'prospect_lb_df':prospect_lb_df, 'prospect_di_df':prospect_di_df, 'prospect_s_df':prospect_s_df}

In [166]:
#Measurements we would like to create our machine learning models off of
var_stats = ['ht', 'wt', 'forty', 'bench', 'vertical', 'broad_jump', 'cone', 'shuttle','grades_defense', 'tackles',
       'assists', 'missed_tackles', 'stops', 'forced_fumbles', 'grades_tackle',
       'grades_run_defense', 'total_pressures', 'hurries', 'hits', 'sacks',
       'grades_pass_rush_defense', 'targets', 'batted_passes', 'interceptions',
       'receptions', 'yards', 'yards_after_catch', 'touchdowns',
       'qb_rating_against', 'grades_coverage_defense', 'grades_defense_last_season', 'tackles_last_season',
       'assists_last_season', 'missed_tackles_last_season',
       'stops_last_season', 'forced_fumbles_last_season',
       'grades_tackle_last_season', 'grades_run_defense_last_season',
       'total_pressures_last_season', 'hurries_last_season',
       'hits_last_season', 'sacks_last_season',
       'grades_pass_rush_defense_last_season', 'targets_last_season',
       'batted_passes_last_season', 'interceptions_last_season',
       'receptions_last_season', 'yards_last_season',
       'yards_after_catch_last_season', 'touchdowns_last_season',
       'qb_rating_against_last_season', 'grades_coverage_defense_last_season', 'power_5_team?']

#Attributes we would like to fill if data is NaN
combine_stats = ['wt', 'forty','bench', 'vertical', 'broad_jump', 'cone', 'shuttle']

for stat in combine_stats:
    prospect_ed_df[stat] = prospect_ed_df[stat].fillna(prospect_s_df[stat].median())

prospect_ed_df

C:\Users\jkepler\AppData\Local\Temp\ipykernel_108500\427717825.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prospect_ed_df[stat] = prospect_ed_df[stat].fillna(prospect_s_df[stat].median())


,season,draft_year,draft_team,draft_round,draft_ovr,pfr_id,cfb_id,player_name,pos,school,...,targets_last_season,batted_passes_last_season,interceptions_last_season,receptions_last_season,yards_last_season,yards_after_catch_last_season,touchdowns_last_season,qb_rating_against_last_season,grades_coverage_defense_last_season,power_5_team?_last_season
79,2016,2016,Undrafted,8.0,256.0,StriEr00,eric-striker-1,Eric Striker,ED,Oklahoma,...,22,4,1,15,115,126,0,57.750000,90.4,1
154,2017,2017,Los Angeles Rams,7.0,234.0,PricEj00,ejuan-price-1,Ejuan Price,ED,Pittsburgh,...,10,2,0,9,51,77,1,116.050000,68.8,1
346,2021,2021,Undrafted,8.0,256.0,HodgDa01,darius-hodge-1,Darius Hodge,ED,Marshall,...,5,1,0,3,12,23,0,21.533333,71.3,0
402,2021,2021,Arizona Cardinals,6.0,210.0,DimuVi00,victor-dimukeje-1,Victor Dimukeje,ED,Duke,...,7,3,1,5,71,72,1,63.025000,67.6,1
406,2021,2021,Undrafted,8.0,256.0,None,ethan-tucky-1,Ethan Tucky,ED,Cincinnati,...,5,2,0,2,9,6,0,31.966667,68.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047,2019,2019,Tampa Bay Buccaneers,4.0,107.0,NelsAn01,anthony-nelson-1,Anthony Nelson,ED,Iowa,...,2,4,0,2,27,17,1,79.166667,68.5,1
1049,2016,2016,Cleveland Browns,3.0,65.0,NassCa00,carl-nassib-1,Carl Nassib,ED,Penn State,...,4,0,1,3,17,16,0,56.300000,66.3,1
1050,2022,2022,Detroit Lions,1.0,2.0,HutcAi00,aidan-hutchinson-1,Aidan Hutchinson,ED,Michigan,...,5,6,0,4,8,16,0,19.800000,77.3,1
1053,2023,2023,Undrafted,8.0,256.0,None,andre-carter-ii-1,Andre Carter II,ED,Army,...,23,1,2,13,135,67,0,36.450000,73.0,0


In [167]:
X = prospect_ed_df[var_stats]
Y = prospect_ed_df['draft_ovr']

In [170]:
model = LinearRegression()
model.fit(X,Y)
y_pred = model.predict(X)
prospect_ed_df['y_pred'] = y_pred
prospect_ed_df.sort_values(by='y_pred')

C:\Users\jkepler\AppData\Local\Temp\ipykernel_108500\2009147344.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prospect_ed_df['y_pred'] = y_pred


,season,draft_year,draft_team,draft_round,draft_ovr,pfr_id,cfb_id,player_name,pos,school,...,batted_passes_last_season,interceptions_last_season,receptions_last_season,yards_last_season,yards_after_catch_last_season,touchdowns_last_season,qb_rating_against_last_season,grades_coverage_defense_last_season,power_5_team?_last_season,y_pred
936,2017,2017,Cleveland Browns,1.0,1.0,GarrMy00,myles-garrett-1,Myles Garrett,ED,Texas A&M,...,3,1,1,13,11,0,39.600000,72.2,1,-56.160024
1022,2020,2020,Washington Football Team,1.0,2.0,YounCh04,chase-young-1,Chase Young,ED,Ohio State,...,6,0,4,10,11,0,54.166667,68.5,1,-15.423704
899,2019,2019,Oakland Raiders,1.0,4.0,FerrCl00,clelin-ferrell-1,Clelin Ferrell,ED,Clemson,...,3,0,7,64,49,0,85.166667,73.6,1,-1.587330
951,2016,2016,San Diego Chargers,1.0,3.0,BosaJo00,joey-bosa-1,Joey Bosa,ED,Ohio State,...,1,1,4,41,18,0,66.300000,65.9,1,5.276221
963,2019,2019,Jacksonville Jaguars,1.0,7.0,AlleJo03,josh-allen-6,Josh Allen,ED,Kentucky,...,2,1,37,338,237,1,81.750000,83.2,1,18.301040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,2017,2017,Undrafted,8.0,256.0,CoxxBr01,bryan-cox-jr-1,Bryan Cox,ED,Florida,...,0,0,1,10,12,0,49.300000,66.0,1,268.964795
895,2023,2023,Undrafted,8.0,256.0,EnecIk00,ikenna-enechukwu-1,Ikenna Enechukwu,ED,Rice,...,0,0,0,0,0,0,9.900000,65.4,0,272.298126
971,2021,2021,Undrafted,8.0,256.0,None,george-silvanic-1,George Silvanic,ED,Air Force,...,0,0,0,0,0,0,0.000000,0.0,0,274.949505
690,2016,2016,Undrafted,8.0,256.0,LittCo01,cory-littleton-1,Cory Littleton,ED,Washington,...,0,0,25,137,179,0,78.100000,78.1,1,285.938604


In [169]:
coefficients = model.coef_
intercept = model.intercept_ 
coefficient_dict = {}

print("Coefficients:")
for i, feature_name in enumerate(X.columns):
    coefficient_dict[feature_name] = coefficients[i]

coefficient_dict['intercept'] = intercept

sorted(coefficient_dict.items(), key=lambda kv_pair: kv_pair[1], reverse=True)

Coefficients:


[('intercept', 1015.2605238512266),
 ('forty', 97.83863314159909),
 ('touchdowns', 17.948622752396474),
 ('touchdowns_last_season', 17.948622752396204),
 ('cone', 17.061074142204692),
 ('forced_fumbles', 1.4091439443813705),
 ('forced_fumbles_last_season', 1.4091439443794034),
 ('missed_tackles', 0.7694387130032294),
 ('missed_tackles_last_season', 0.7694387130016742),
 ('assists', 0.7468130997527176),
 ('assists_last_season', 0.7468130997507849),
 ('hurries', 0.5833850915583838),
 ('hurries_last_season', 0.5833850915560745),
 ('stops_last_season', 0.5067291777620362),
 ('stops', 0.50672917776199),
 ('yards_after_catch', 0.4803547339854459),
 ('yards_after_catch_last_season', 0.48035473398223694),
 ('grades_coverage_defense', 0.4728667268550968),
 ('grades_coverage_defense_last_season', 0.47286672685473263),
 ('grades_tackle', 0.3607271552402338),
 ('grades_tackle_last_season', 0.36072715523906496),
 ('batted_passes', 0.29219654525248906),
 ('batted_passes_last_season', 0.2921965452512